In [1]:
import os

import web3
from web3 import Web3
import pandas as pd
from tqdm import tqdm
from decouple import config

In [2]:
w3 = Web3(Web3.HTTPProvider(f"{config('MAINNET_ACCESS_URL')}", request_kwargs={'timeout':60}))
address = Web3.toChecksumAddress("0x0ba45a8b5d5575935b8158a88c631e9f9c95a2e5")
path = os.path.join('..', 'abi', 'abi.json')
with open(path) as f:
    abi = f.read()
old_tellor = w3.eth.contract(address=address, abi=abi)

In [34]:
#Read binance balances csv
df = pd.read_csv('../data/Tellor3- todo fix - Hotbit.csv')
df = df.drop([61, 62], axis=0)
print(df.shape)
df.head()

(61, 2)


,address,balance_reported
0,0xe39d7e6ef9263e8e53de0a2497c4dbb4716a2993,0.000100
1,0x76d0f439fa25fc4bd37062c4ad680128176ee2df,0.001279
2,0x5063838dafd492f77770850bfd506ae06c2d123c,0.002600
3,0x24a21c8fcac31ae67373b6ce7d46e8d69815b1fd,0.008019
4,0x4e290f3f60c1110636bea1637032cef9772cea1a,0.009135


In [30]:
df.isna().sum()

address             0
balance_reported    0
dtype: int64

In [31]:
def check_balance(address):
    address = Web3.toChecksumAddress(address)
    return old_tellor.functions.balanceOf(address).call()

check_balance('0xe39d7e6ef9263e8e53de0a2497c4dbb4716a2993')

100000000000000

In [36]:
df["balance_validated"] = df["address"].apply(check_balance)
df["balance_validated"].head()

0     100000000000000
1    1279250000000000
2    2600000000000000
3    8019450000000000
4    9134810000000000
Name: balance_validated, dtype: object

In [37]:
df["balance_validated"] = df["balance_validated"] / 1E18
df["balance_validated"].head()

0    0.000100
1    0.001279
2    0.002600
3    0.008019
4    0.009135
Name: balance_validated, dtype: object

In [40]:
df['balances_match'] = df["balance_validated"] == df['balance_reported']
print(df['balances_match'].value_counts())
df.head()

True     51
False    10
Name: balances_match, dtype: int64


,address,balance_reported,balance_validated,balances_match
0,0xe39d7e6ef9263e8e53de0a2497c4dbb4716a2993,0.000100,0.000100,True
1,0x76d0f439fa25fc4bd37062c4ad680128176ee2df,0.001279,0.001279,True
2,0x5063838dafd492f77770850bfd506ae06c2d123c,0.002600,0.002600,True
3,0x24a21c8fcac31ae67373b6ce7d46e8d69815b1fd,0.008019,0.008019,True
4,0x4e290f3f60c1110636bea1637032cef9772cea1a,0.009135,0.009135,True


In [42]:
df['diff'] = df["balance_validated"] - df['balance_reported']
df['diff'].sort_values()

60   -0.000001
58   -0.000000
31   -0.000000
21   -0.000000
55   -0.000000
        ...   
18    0.000000
59    0.000000
12    0.000000
16    0.000000
53    0.000000
Name: diff, Length: 61, dtype: object

In [43]:
df.to_csv('../data/Tellor3-Tally-Hotbit.csv')